<a href="https://colab.research.google.com/github/pravallika-1305/WISE_ML/blob/main/RNN_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# load ascii text and covert to lowercase
filename = "/content/drive/My Drive/Data/wonderland.txt"
raw_text = open(filename, 'r', encoding='utf-8').read()
raw_text = raw_text.lower()



In [4]:
# create mapping of unique chars to integers
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))

In [6]:
n_chars = len(raw_text)
n_vocab = len(chars)
print("Total Characters: ", n_chars)
print("Total Vocab: ", n_vocab)

Total Characters:  164045
Total Vocab:  63


In [7]:
# prepare the dataset of input to output pairs encoded as integers
seq_length = 100
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
	seq_in = raw_text[i:i + seq_length]
	seq_out = raw_text[i + seq_length]
	dataX.append([char_to_int[char] for char in seq_in])
	dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print("Total Patterns: ", n_patterns)

Total Patterns:  163945


In [8]:

# reshape X to be [samples, time steps, features]
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))
# normalize
X = X / float(n_vocab)
# one hot encode the output variable
y = np_utils.to_categorical(dataY)


In [9]:
# define the LSTM model
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [11]:
filepath="weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [12]:
model.fit(X, y, epochs=20, batch_size=128,callbacks=callbacks_list)

Epoch 1/20
1281/1281 [==============================] - 846s 661ms/step - loss: 2.9680

Epoch 00001: loss improved from inf to 2.96797, saving model to weights-improvement-01-2.9680.hdf5
Epoch 2/20
1281/1281 [==============================] - 839s 655ms/step - loss: 2.8388

Epoch 00002: loss improved from 2.96797 to 2.83878, saving model to weights-improvement-02-2.8388.hdf5
Epoch 3/20
1281/1281 [==============================] - 841s 656ms/step - loss: 2.7668

Epoch 00003: loss improved from 2.83878 to 2.76677, saving model to weights-improvement-03-2.7668.hdf5
Epoch 4/20
1281/1281 [==============================] - 837s 654ms/step - loss: 2.7016

Epoch 00004: loss improved from 2.76677 to 2.70156, saving model to weights-improvement-04-2.7016.hdf5
Epoch 5/20
1281/1281 [==============================] - 838s 654ms/step - loss: 2.6468

Epoch 00005: loss improved from 2.70156 to 2.64677, saving model to weights-improvement-05-2.6468.hdf5
Epoch 6/20
1281/1281 [===========================

In [14]:
# pick a random seed
import sys



int_to_char = dict((i, c) for i, c in enumerate(chars))
start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print("Seed:")
print("\"", ''.join([int_to_char[value] for value in pattern]), "\"")
# generate characters
for i in range(1000):
	x = numpy.reshape(pattern, (1, len(pattern), 1))
	x = x / float(n_vocab)
	prediction = model.predict(x, verbose=0)
	index = numpy.argmax(prediction)
	result = int_to_char[index]
	seq_in = [int_to_char[value] for value in pattern]
	sys.stdout.write(result)
	pattern.append(index)
	pattern = pattern[1:len(pattern)]
print("\nDone.")

Seed:
" _ mind what you’re doing!” cried alice, jumping up and down
in an agony of terror. “oh, there goes h "
ate an at the waite to tee the oi the saae to tee sie of the saaei, and the waited to the toeee an ih shu so the tabde hf i soede to tee the was oo the tooe. “he ioer tht toen i taad to the kante oo toe toe oe the saaei of the sooe. 
“h woul io ”hu ”hul to tou then you mane,” said the daterpillar.

“whll, i shanl you dane to tee woine io the woudd ”ou ”hul to tou,” 
“i wonl you dan ”ou dane wo the woide of the soie,” saed the gatter. “ih sou de io whe hors of the goor ”f
thee io the soees ”ou thu      bn af iod tee th the wait oo to the wai oo the tooe. 
“h whnl toe ”ou donw ”hu      bn of the sooe tf the woile.”

“i wonl you can ”ou dane wo the coolo ” saed the daterpillar.

“ie cours an ier ”as _ lirt ei a larter ” said alice. “ih aod toe karte oo tee toiee of the saae to tee siee ”ou  an you dan tou donw ”hut toe oo toen i whol tou ”hu      bn at aad to de      ani the woid